In [3]:
import requests
import json
import os
import dvc.api
import json
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import transform_bounds
from tqdm import tqdm
import concurrent.futures
from datetime import datetime


params = dvc.api.params_show()

def save_json(data, directory = params['paths']['satellite']):
    if not os.path.exists(directory):
        os.makedirs(directory)

    for feature in data:
        feature_id = feature['id']
        filename = os.path.join(os.path.abspath(directory), f"{feature_id}.json")
        with open(filename, 'w') as file:
            json.dump(feature, file)
            print(f"Saved {filename}")


url = 'https://earth-search.aws.element84.com/v1/search'
headers = {
    'authority': 'earth-search.aws.element84.com',
    'accept': 'application/geo+json',
    'accept-language': 'en,de;q=0.7,en-US;q=0.7,de-CH;q=0.7,es;q=0.2,fr;q=0.2,it;q=0.2,ro;q=0.2',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'dnt': '1',
    'origin': 'https://radiantearth.github.io',
    'pragma': 'no-cache',
    'referer': 'https://radiantearth.github.io/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

Now we save all the meta data for the images in JSON format.

In [2]:
bbox = params['zueri_crop']['bbox']

initial_body = {
    'datetime': params['search_dates'],
    'bbox': bbox,
    'limit': 16,
    'collections': ['sentinel-2-l2a']
}

In [3]:

response = requests.post(url, headers=headers, json=initial_body)

save_json(response.json()['features'])
next_body = response.json()['links'][0]['body']
while next_body:
    response = requests.post(url, headers=headers, json=next_body)
    data = response.json()
    save_json(data['features'])
    next_body = data['links'][0]['body'] \
        if 'links' in data \
            and len(data['links']) > 0 \
            and data['links'][0]['rel'] == 'next' \
        else None


Saved c:\dev\messis\data\data\satellite\S2B_32TMT_20190929_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2B_32TNT_20190929_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TMT_20190927_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TNT_20190927_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TMT_20190924_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TNT_20190924_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2B_32TMT_20190922_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2B_32TNT_20190922_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2B_32TMT_20190919_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2B_32TNT_20190919_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TMT_20190917_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TNT_20190917_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TMT_20190914_0_L2A.json
Saved c:\dev\messis\data\data\satellite\S2A_32TNT_20190914_0_L2A.json
Saved c:\dev\messis\

In [4]:
# read all the json files and check whether their geometry fulfills the bbox and only keep the ones that do and are the latest version


bbox_shape = Polygon([
    (bbox[0], bbox[1]),
    (bbox[0], bbox[3]),
    (bbox[2], bbox[3]),
    (bbox[2], bbox[1]),
    (bbox[0], bbox[1])
])

def read_jsons(directory = params['paths']['satellite']):
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            with open(os.path.join(directory, filename), 'r') as file:
                yield json.load(file)

debug = False
matches = 0
def check_bbox(feature, bbox):
    geometry = feature['geometry']
    if len(geometry['coordinates']) != 1:
        if debug:
            print(f"Feature {feature['id']} has more than one polygon")
        return False
    coverage = Polygon(geometry['coordinates'][0])
    if debug:
        # plot the coverage and the bbox (BLUE is ZüriCrop, RED is the coverage)
        fig, ax = plt.subplots()
        x, y = coverage.exterior.xy
        ax.plot(x, y, color='red')
        x, y = bbox.exterior.xy
        ax.plot(x, y, color='blue')
        plt.show()
    if not coverage.intersects(bbox):
        if debug:
            print(f"Feature {feature['id']} does not intersect the bbox")
        return False
    elif not coverage.contains(bbox):
        if debug:
            print(f"Feature {feature['id']} does not contain the bbox")
        return False
    if feature['properties']['eo:cloud_cover'] > params['max_cloud_cover']:
        if debug:
            print(f"Feature {feature['id']} has too high cloud cover")
        return False
    return True

files_to_keep = []
for feature in read_jsons():
    matched = check_bbox(feature, bbox_shape)
    if matched:
        matches += 1
        files_to_keep.append(feature['id'] + ".json")


print(f"Found {matches} matches")

# for the remaining files, there are some duplicates: 
# S2A_32TNT_20200603_0_L2A.json
# S2A_32TNT_20200603_1_L2A.json
        
# or S2B_32TMT_20200817_1_L2A.json
# S2B_32TMT_20200817_3_L2A.json
# delete the ones that are not the latest version
files_grouped = {}
for filename in files_to_keep:
    parts = filename.split("_")
    key = parts[2]
    if key not in files_grouped:
        files_grouped[key] = []
    files_grouped[key].append(filename)

files_to_keep = []
for key in files_grouped.keys():
    if len(files_grouped[key]) > 1:
        print(f"Found multiple files for {key}: {files_grouped[key]}")
        files_grouped[key].sort()
        files_to_keep.append(files_grouped[key][-1])
    else:
        files_to_keep.append(files_grouped[key][0])

print(f"Found {len(files_to_keep)} matches")

Found 16 matches
Found 16 matches


In [6]:
# pick the first file and the last file and the file in the middle
files_to_download = []
files_to_download.append(files_to_keep[0])
files_to_download.append(files_to_keep[-1])
# extract the date from the filenames and determine the difference in days
dates = []
for filename in files_to_download:
    parts = filename.split("_")
    date = parts[2]
    dates.append(date)
dates.sort()
date1 = files_to_download[0].split("_")[2]
date3 = files_to_download[-1].split("_")[2]

date1 = datetime.strptime(date1, "%Y%m%d")
date3 = datetime.strptime(date3, "%Y%m%d")

middate = date1 + (date3 - date1) / 2

# create a list with absolute differences between the middate and the dates of the remaining files
diffs = []
for filename in files_to_keep:
    parts = filename.split("_")
    date = parts[2]
    date = datetime.strptime(date, "%Y%m%d")
    diffs.append(abs(middate - date).days)

# find the file with the smallest difference
index = diffs.index(min(diffs))
# ensure index is not first or last
assert index != 0
assert index != len(files_to_keep) - 1
print(f"diffs: {min(diffs)}, index: {index}")
files_to_download.insert(1, files_to_keep[index])


print(f"Downloading images for {files_to_download}")

date1 = datetime.strptime(files_to_download[0].split("_")[2], "%Y%m%d")
date2 = datetime.strptime(files_to_download[1].split("_")[2], "%Y%m%d")
date3 = datetime.strptime(files_to_download[2].split("_")[2], "%Y%m%d")

print(f"Days between the first and second image: {(date2 - date1).days}")
print(f"Days between the second and third image: {(date3 - date2).days}")


diffs: 1, index: 3
Days between 2019-03-21 00:00:00 and 2019-06-26 00:00:00: 97
Days between 2019-06-26 00:00:00 and 2019-09-29 00:00:00: 95


In [7]:

def download_geotiff_cog(url, bbox, filename):
    # check if file already exists
    if os.path.exists(filename):
        return
    with rasterio.open(url) as src:
        bbox = transform_bounds('EPSG:4326', src.crs, *bbox)
        window = src.window(*bbox)
        subset = src.read(window=window)

        if not os.path.exists(params['paths']['satellite']):
            os.makedirs(params['paths']['satellite'])

        with rasterio.open(filename,
                           'w',
                           driver='GTiff',
                           width=window.width,
                           height=window.height,
                           count=src.count,
                           dtype=src.dtypes[0],
                           crs=src.crs,
                           transform=src.window_transform(window)) as dst:
            dst.write(subset)

download_tasks = []

for filename in files_to_download:
    with open(os.path.join(params['paths']['satellite'], filename), 'r') as file:
        feature = json.load(file)
        assets = feature['assets']
        id = feature['id']
        bands = ['red', 'green', 'blue', 'nir', 'swir16', 'swir22']
        for band in bands:
            if band not in assets:
                print(f"Band {band} is missing for {id}")
                continue
            url = assets[band]['href']
            download_tasks.append((url, id, band))

def download_task(task):
    url, id, band = task
    download_geotiff_cog(url, bbox, os.path.join(params['paths']['satellite'], f"{id}_{band}.tif"))

# Define a function to report progress
def update_progress(future):
    pbar.update(1)

# Set up a progress bar
with tqdm(total=len(download_tasks), desc="Downloading GeoTIFF COGs") as pbar:
    # Use ThreadPoolExecutor to run download tasks in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # Submit download tasks to executor
        futures = [executor.submit(download_task, task) for task in download_tasks]
        # Add progress update callback for each future
        for future in concurrent.futures.as_completed(futures):
            future.add_done_callback(update_progress)

KeyboardInterrupt: 